<img src="./pictures/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Frame selection
*Written by Marc Budinger, INSA Toulouse, France*

## Design graph 

The following diagram represents the design graph of the frame selection. 


![DesignGraph](pictures/FrameDesignGraph.png)

> **Questions:**
* Give the main sizing problems you are able to detect.
* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of the frame.


In [1]:
from utils.model_standard import CoreModel
from utils.model_serializer import ModelSerializer

In [2]:
import math
from math import pi
from math import sin
import scipy

class FrameModel(CoreModel):
    """
    Frame model class.
    ----------
    """

    def __init__(self, **kwargs):
        super(FrameModel, self).__init__(**kwargs)
        self.initialization()
        self.execute()
        self._update()

    def initialization(self):
        
        # Input variables
        inputs = {'Narm': 4.0, 'Dpro': 1.0, 'Np_arm': 1.0, 'Tpro_takeoff': 15.0, 'k_frame': 0.1}
        self.set_inputs(inputs)
        
        # Input parameters
        
        # Static stress
        # Sigma_max=200e6/4 # [Pa] Alu max stress (2 reduction for dynamic, 2 reduction for stress concentration)
        Sigma_max=280e6/4 # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
        
        inputs = {'Sigma_max': Sigma_max}
        self.set_inputs(inputs)
        
        # Declare outputs
        outputs = ['sep', 'Lb', 'Dfra', 'Efra', 'Mfra']   
        self.declare_outputs(outputs)

    def execute(self):
        
        # Get input values
        Narm, Dpro, Np_arm, Tpro_takeoff, k_frame = self.get_values(['Narm', 'Dpro', 'Np_arm', 'Tpro_takeoff', 'k_frame'])
        Sigma_max = self.get_values(['Sigma_max'])

        #######
        
        Lb = Dpro/(2*scipy.sin(pi/Narm))
        
        d = ((Tpro_takeoff*Np_arm*Lb*0.5*(1/k_frame))/(pi/64 * Sigma_max*((1/k_frame)**4 - 1)))**(1/3)  #diametre interieur du tube
             
        Dfra = d / k_frame
        Efra = (Dfra - d)/2
        Vfra = Lb*pi/4*(Dfra**2 - d**2)  #Volume tube
        
        Mfra = Vfra*1700 #rho_composite = 1700 kg/m**3
        sep = 2*pi/Narm
        
        #######
    
        outputs = {'sep': sep, 'Lb': Lb, 'Dfra': Dfra, 'Efra': Efra, 'Mfra': Mfra}
        self.set_outputs(outputs)
                
    def __str__(self):
        
        s =(("* Frame informations: \n") +
            ("** Global: \n") + 
            ("    Frame mass = %.2f kg" %(self.get_values(['Mfra'])) + "\n") +
            ("** Geometry: \n") +
            ("    Beam diameter = %.2f mm" %(self.get_values(['Dfra'])*1000) + "\n") + 
            ("    Thickness = %.2f mm" % (self.get_values(['Efra'])*1000) + "\n") + 
            ("    Length of the arm = %.2f mm" % (self.get_values(['Lb'])*1000) + "\n") +
            ("    Interior angle / separation between propellers = %.2f °" % (self.get_values(['sep'])*180/pi)+ "\n")
           )
        return s
        

In [3]:
frame_model = FrameModel()

print(frame_model)

ms = ModelSerializer()
path = './models_student/'
file_name = 'frame_model'
ms.save_model(frame_model, path + file_name)

* Frame informations: 
** Global: 
    Frame mass = 0.12 kg
** Geometry: 
    Beam diameter = 11.56 mm
    Thickness = 5.20 mm
    Length of the arm = 707.11 mm
    Interior angle / separation between propellers = 90.00 °



In [4]:
print(frame_model)

inputs = {'Tpro_takeoff': 25.0}

frame_model.evaluate(inputs, [])

print(frame_model)

* Frame informations: 
** Global: 
    Frame mass = 0.12 kg
** Geometry: 
    Beam diameter = 11.56 mm
    Thickness = 5.20 mm
    Length of the arm = 707.11 mm
    Interior angle / separation between propellers = 90.00 °

* Frame informations: 
** Global: 
    Frame mass = 0.18 kg
** Geometry: 
    Beam diameter = 13.70 mm
    Thickness = 6.17 mm
    Length of the arm = 707.11 mm
    Interior angle / separation between propellers = 90.00 °



### Commentaires sur le frame : 
On peut voir que la masse de la structure est très faible et ne représentera pas un un fort pourcentage de la masse totale du drone. Cela laisse à penser qu'un gain de masse sur la structure, par exemple via un nouveau matériau composite plus léger, serait certe bénéfique pour la masse totale du drone mais n'aurait pas un impact déterminant. Il est sûrement plus intéressant de concentrer les efforts de développement sur les batteries.


- la poussée de l'hélice et le poids vont dimensionner les contraintes en flexion des poutres de la structure ; on arrive donc à un compromis pour la masse en considérant efforts et contraintes
- la phase dimensionnante est celle de forte accélération, donc de take-off
- on prend un coefficient de sécurité pour le dimensionnement en fatigue
- pour les vibrations : il ne faut pas exciter à une fréquence de résonnance de l'airframe avec la fréquence du moteur, qui doivent rester différentes pour ne pas avoir de phénomène vibratoire
